<a href="https://colab.research.google.com/github/Bayzid03/LangGraph-Hub/blob/main/AI%20News%20Summarizer/Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from typing import TypedDict, Annotated, List, Dict
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables.graph import MermaidDrawMethod
from Ipython.display import display, Image
from dotenv import load_dotenv
import requests
import json
from datetime import datetime

In [ ]:
# load environment variables

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-05-20", temperature=0.3)

Define Agent State and Prompts

In [ ]:
class NewsSummarizerState(TypedDict):
  messages: Annotated[List[HumanMessage | AIMessage], "The messages in the conversation"]
  search_query: str
  search_filter: Dict[str, str]
  raw_news_data: List[Dict]
  processed_articles: List[Dict]
  summary_style: str
  final_summary: str

news_search_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a news search query optimizer. Convert the user's request into an optimized search query for finding relevant news articles. Make it concise and focused."),
    ("human", "User request: {user_input}\nOptimize this into a focused news search query."),
])

news_filter_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a news filtering assistant. Based on the user's request, determine appropriate filters for news search. Return a JSON with 'days' (1-30), 'max_results' (5-20), and 'include_domains' (empty list if no preference)."),
    ("human", "User request: {user_input}\nProvide search filters as JSON."),
])

news_summarize_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert news summarizer. Create a comprehensive summary of the provided news articles in {summary_style} style.

Summary styles:
- brief: 2-3 sentences per article, focus on key points
- detailed: Full paragraph per article with context and implications
- bullet: Key points in bullet format
- executive: Business-focused summary with impact analysis

Structure your response as:
**NEWS SUMMARY - {search_query}**
**Generated on: {timestamp}**

**KEY HIGHLIGHTS:**
[Overall themes and important developments]

**ARTICLE SUMMARIES:**
[Individual article summaries based on style]

**SOURCES:**
[List of sources with publication dates]"""),
    ("human", "Summarize these news articles:\n\n{articles_text}"),
])


Define Agent Functions (Nodes)

In [ ]:
def input_search_query(state: NewsSummarizerState) -> NewsSummarizerState:
    """
    Node 1: Get search query from user and optimize it using LLM
    """
    print("📰 Welcome to AI News Summarizer!")
    print("Please enter your news topic or search query:")
    print("Examples: 'AI developments', 'climate change policy', 'tech earnings', 'global economy'")
    user_message = input("Enter your search query:")

    response = llm.invoke(news_search_prompt.format_messages(user_input=user_message))
    query = response.content.strip()

    return{
        **state,
        "search_query": query,
        "messages": state["messages"] + [HumanMessage(content=user_message)]
    }

def input_search_filters(state: NewsSummarizerState) -> NewsSummarizerState:
    """
    Node 2: Get search filters and preferences from user
    """
    print("Please specify your preferences:")

    # Get summary style
    print("\nSummary style options: brief, detailed, bullet, executive")
    style = input("Choose summary style (default: brief):").strip().lower() or "brief"

    # Get time range
    print("\nTime range options: 1 (today), 3 (last 3 days), 7 (last week), 30 (last month)")
    days = input("Days to search (default: 7):").strip() or "7"

    # Get number of articles
    print("\nNumber of articles: 5-20")
    max_results = input("Max articles to analyze (default: 10): ").strip() or "10"

    filters = {
        "days": int(days),
        "max_results": int(max_results),
        "include_domains": []
    }

    print(f"\n📋 Filters set: {filters}")

    return {
        **state,
        "search_filters": filters,
        "summary_style": style,
        "messages": state['messages'] + [HumanMessage(content=f"Filters: {filters}, Style: {style}")],
    }

def fetch_news_articles(state: NewsSummarizerState) -> NewsSummarizerState:
    print(f"\n🔎 Searching for news articles about '{state['search_query']}'...")

    # Tavily API endpoint
    tavily_url = "https://api.tavily.com/search"

    # Prepare request payload
    payload = {
        "api_key": TAVILY_API_KEY,
        "query": state["search_query"],
        "search_depth": "advanced",
        "include_answer": False,
        "include_raw_content": True,
        "max_results": state["search_filters"]["max_results"],
        "include_domains": state["search_filters"]["include_domains"],
        "exclude_domains": ["youtube.com", "twitter.com", "facebook.com"],
        "days": state["search_filters"]["days"],
    }

    try:
        # Send POST request to Tavily
        response = requests.post(
            tavily_url,
            json=payload,
            headers={"Content-Type": "application/json"}
        )
        response.raise_for_status()
        search_results = response.json()

        # Extract articles from response
        articles = search_results.get("results", [])
        print(f"✅ Found {len(articles)} articles")

        # Process and reformat articles
        processed_articles = []
        for article in articles:
            processed_article = {
                "title": article.get("title", "No Title"),
                "url": article.get("url", ""),
                "content": article.get("content", ""),
                "raw_content": article.get("raw_content", ""),
                "published_date": article.get("published_date", "Unknown"),
                "score": article.get("score", 0)
            }
            processed_articles.append(processed_article)
            print(f"📄 {processed_article['title'][:60]}...")

        print("\n" + "=" * 50 + "\n")

        return {
            **state,
            "raw_news_data": articles,
            "processed_articles": processed_articles,
            "messages": state["messages"] + [AIMessage(content=f"Found {len(articles)} articles")],
        }

    except Exception as e:
        print(f"❌ Error fetching news: {str(e)}")

        # Fallback sample article
        mock_articles = [{
            "title": f"Sample article about {state['search_query']}",
            "content": f"This is sample content related to {state['search_query']} for demonstration purposes.",
            "url": "https://example.com",
            "published_date": datetime.now().strftime("%Y-%m-%d"),
            "score": 0.8
        }]

        return {
            **state,
            "raw_news_data": mock_articles,
            "processed_articles": mock_articles,
            "messages": state["messages"] + [AIMessage(content="Using sample data due to API error")],
        }

def summarize_news(state: NewsSummarizerState) -> NewsSummarizerState:
    """
    Node 4: Generate AI-powered summary of fetched news articles
    """
    print(f"📝 Creating {state['summary_style']} summary of news articles...")
    print(f"Processing {len(state['processed_articles'])} articles...\n")

    # Prepare articles text for summarization
    articles_text = ""
    for i, article in enumerate(state['processed_articles'], 1):
        content = article.get('raw_content', article.get('content', ''))
        articles_text += f"Article {i}:\n"
        articles_text += f"Title: {article['title']}\n"
        articles_text += f"URL: {article['url']}\n"
        articles_text += f"Published: {article.get('published_date', 'Unknown')}\n"
        articles_text += f"Content: {content[:1000]}...\n\n"  # Limit content length

    # Generate summary using LLM
    response = llm.invoke(news_summarize_prompt.format_messages(
        summary_style=state['summary_style'],
        search_query=state['search_query'],
        timestamp=datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        articles_text=articles_text
    ))

    summary = response.content

    print("📊 Your AI-Generated News Summary:")
    print("="*60)
    print(summary)
    print("="*60)

    return {
        **state,
        "final_summary": summary,
        "messages": state['messages'] + [AIMessage(content=summary)],
    }

GRAPH CREATION AND COMPILATION

In [ ]:
  workflow = StateGraph(NewsSummarizerState)

  workflow.add_node("input_search_query", input_search_query)
  workflow.add_node("input_search_filters", input_search_filters)
  workflow.add_node("fetch_news_articles", fetch_news_articles)
  workflow.add_node("summarize_news", summarize_news)

  workflow.set_entry_point("input_search_query")

  workflow.add_edge("input_search_query", "input_search_filters")
  workflow.add_edge("input_search_filters", "fetch_news_articles")
  workflow.add_edge("fetch_news_articles", "summarize_news")
  workflow.add_edge("summarize_news",END)

  app = workflow.compile()

In [ ]:
display(Image(app.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.API)))

In [ ]:
# Define the function that runs the graph
def run_news_summarizer(user_request: str):
    print("=" * 50)
    print("🤖 AI News Summarizer Starting...")
    print(f"Initial Request: {user_request}\n")
    print("=" * 50)

    state = {
        "messages": [HumanMessage(content=user_request)],
        "search_query": "",
        "search_filters": {},
        "raw_news_data": [],
        "processed_articles": [],
        "summary_style": "brief",
        "final_summary": "",
    }

    for output in app.stream(state):
        pass  # The nodes themselves handle all printing

    print("\n🎉 News summarization completed! Stay informed!")

if __name__ == "__main__":
    user_request = "I want to get a summary of the latest news."
    run_news_summarizer(user_request)